In [157]:
import pandas as pd
import warnings
import matplotlib.pyplot as plt
%matplotlib inline
import mglearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler  
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.manifold import TSNE
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
from sklearn import metrics
import datetime
from alpha_vantage.timeseries import TimeSeries
from pprint import pprint
import ta
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.linear_model import Lasso
import warnings
from operator import itemgetter 
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

In [80]:
df = pd.read_csv("/Users/rhonanbennett/Desktop/2/Uni/CBS/Business Analyticis /Others/AirBnb/listings.csv")
df = pd.DataFrame(df)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28418 entries, 0 to 28417
Data columns (total 16 columns):
id                                28418 non-null int64
name                              28358 non-null object
host_id                           28418 non-null int64
host_name                         28408 non-null object
neighbourhood_group               0 non-null float64
neighbourhood                     28418 non-null object
latitude                          28418 non-null float64
longitude                         28418 non-null float64
room_type                         28418 non-null object
price                             28418 non-null int64
minimum_nights                    28418 non-null int64
number_of_reviews                 28418 non-null int64
last_review                       23768 non-null object
reviews_per_month                 23768 non-null float64
calculated_host_listings_count    28418 non-null int64
availability_365                  28418 non-null int64
dt

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,6983,Copenhagen 'N Livin',16774,Simon,NaN,Nrrebro,55.68798,12.54571,Private room,373,2,168,2019-07-19,1.32,1,0
1,26057,Lovely house - most attractive area,109777,Kari,NaN,Indre By,55.69163,12.57459,Entire home/apt,2397,3,50,2019-12-14,0.66,1,333
2,26473,City Centre Townhouse Sleeps 1-12 persons,112210,Oliver,NaN,Indre By,55.67590,12.57698,Entire home/apt,3103,3,292,2020-02-21,2.56,2,81
3,29118,Best Location in Cool Istedgade,125230,Nana,NaN,Vesterbro-Kongens Enghave,55.67069,12.55430,Entire home/apt,801,7,22,2019-08-02,0.19,1,26
4,29618,Artsy and familyfriendly home in lovely Copenh...,127577,Simon And Anna,NaN,sterbro,55.69375,12.56945,Entire home/apt,862,7,90,2017-06-03,0.77,1,0


In [81]:
for col in df:
    if(df[col].isnull().any()):
        print(col)

name
host_name
neighbourhood_group
last_review
reviews_per_month


In [82]:
# removing the irrelevant columns
cols_to_drop = ["id","name","host_id", "host_name", "neighbourhood_group", 
               "minimum_nights", "number_of_reviews", "last_review", "reviews_per_month",
               "calculated_host_listings_count", "availability_365"]
df = df.drop(columns=cols_to_drop,axis=1)

df.head()



,neighbourhood,latitude,longitude,room_type,price
0,Nrrebro,55.68798,12.54571,Private room,373
1,Indre By,55.69163,12.57459,Entire home/apt,2397
2,Indre By,55.67590,12.57698,Entire home/apt,3103
3,Vesterbro-Kongens Enghave,55.67069,12.55430,Entire home/apt,801
4,sterbro,55.69375,12.56945,Entire home/apt,862


In [83]:
for col in df:
    if(df[col].isnull().any()):
        print(col)

In [84]:
numerical_columns = [col for col in df.columns if (df[col].dtype=='int64' or df[col].dtype=='float64') 
                     and col != 'Exited']

df[numerical_columns].describe().loc[['min','max', 'mean','50%'],:]

,latitude,longitude,price
min,55.615930,12.452090,0.00000
max,55.733710,12.639800,149453.00000
mean,55.680777,12.557817,855.38736
50%,55.682300,12.554550,703.00000


In [85]:
df[df['price'] == df['price'].min()]

,neighbourhood,latitude,longitude,room_type,price
13801,Vesterbro-Kongens Enghave,55.67082,12.54052,Entire home/apt,0
15222,Valby,55.66609,12.50310,Entire home/apt,0
15436,Indre By,55.67819,12.58269,Entire home/apt,0


In [86]:
df_new = df.drop(index=[13801, 15222, 15436])


In [87]:
df_new[numerical_columns].describe().loc[['min','max', 'mean','50%'],:]

,latitude,longitude,price
min,55.615930,12.452090,68.00000
max,55.733710,12.639800,149453.00000
mean,55.680778,12.557819,855.47767
50%,55.682300,12.554550,703.00000


In [88]:
df_new[df_new['price'] == df_new['price'].min()]

,neighbourhood,latitude,longitude,room_type,price
1720,Vesterbro-Kongens Enghave,55.66777,12.55378,Entire home/apt,68
9093,Vesterbro-Kongens Enghave,55.67247,12.54990,Private room,68
10043,Amager st,55.65681,12.63357,Entire home/apt,68
17916,Vesterbro-Kongens Enghave,55.65456,12.55275,Entire home/apt,68


In [131]:
df_new[df_new['price'] == df_new['price'].max()]

,neighbourhood,latitude,longitude,room_type,price
26085,Amager st,55.66418,12.62204,Entire home/apt,149453


In [132]:
df_new.to_csv("new2.csv")

In [133]:
df_new = df = pd.read_csv('new2.csv', index_col = 0)
df_new.info()
df_new.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28415 entries, 0 to 28417
Data columns (total 5 columns):
neighbourhood    28415 non-null object
latitude         28415 non-null float64
longitude        28415 non-null float64
room_type        28415 non-null object
price            28415 non-null int64
dtypes: float64(2), int64(1), object(2)
memory usage: 1.3+ MB


,neighbourhood,latitude,longitude,room_type,price
0,Nrrebro,55.68798,12.54571,Private room,373
1,Indre By,55.69163,12.57459,Entire home/apt,2397
2,Indre By,55.67590,12.57698,Entire home/apt,3103
3,Vesterbro-Kongens Enghave,55.67069,12.55430,Entire home/apt,801
4,sterbro,55.69375,12.56945,Entire home/apt,862


In [158]:
from sklearn.preprocessing import MinMaxScaler

Scaler = MinMaxScaler()

homes = pd.read_csv("new2.csv", index_col = 0)
homes.head()




,neighbourhood,latitude,longitude,room_type,price
0,Nrrebro,55.68798,12.54571,Private room,373
1,Indre By,55.69163,12.57459,Entire home/apt,2397
2,Indre By,55.67590,12.57698,Entire home/apt,3103
3,Vesterbro-Kongens Enghave,55.67069,12.55430,Entire home/apt,801
4,sterbro,55.69375,12.56945,Entire home/apt,862


In [159]:
cols_to_drop = ["neighbourhood", "room_type" ]
df_free = homes.drop(columns=cols_to_drop,axis=1)

df_free.head()

,latitude,longitude,price
0,55.68798,12.54571,373
1,55.69163,12.57459,2397
2,55.67590,12.57698,3103
3,55.67069,12.55430,801
4,55.69375,12.56945,862


In [160]:
x = df_free[['latitude', 'longitude']]
y = df_free['price']
df_free.shape

(28415, 3)

In [161]:
from sklearn.preprocessing import MinMaxScaler

Scaler = MinMaxScaler(df_free)



In [165]:
x = df_free[['latitude', 'longitude']]
y = df_free['price']
x_train_dum, x_test_dum, y_train_dum, y_test_dum = train_test_split(x, y,  test_size = 0.2, random_state = 1)

print (x_train_dum.shape, y_train_dum.shape)
print (x_test_dum.shape, y_test_dum.shape)

Lin_dum = LinearRegression()
Lin_dum.fit(x_train_dum, y_train_dum)

print (Lin_dum)
print ("Lin_dum train score:", Lin_dum.score (x_train_dum, y_train_dum, sample_weight = None))
print ("Lin_dum test score:", Lin_dum.score (x_test_dum, y_test_dum, sample_weight = None))
print(cross_val_score(Lin_dum, x, y, cv=3))

(22732, 2) (22732,)
(5683, 2) (5683,)
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)
Lin_dum train score: 0.005349847378593786
Lin_dum test score: 0.004667259689086434
[ 0.00924055 -0.0089239  -0.00027436]


In [155]:
training_accuracy_ran = dict()
test_accuracy_ran = dict()

max_features_settings = range(1,21) 

for max_features in max_features_settings:
    key = str(max_features)
    ran_forest_ran = RandomForestClassifier(n_estimators=max_features, random_state=0)
    ran_forest_ran.fit(x_train_dum, y_train_dum)
    
    value = ran_forest_ran.score(x_train_dum, y_train_dum)
    training_accuracy_ran[key] = value
    
    value = ran_forest_ran.score(x_test_dum, y_test_dum)
    test_accuracy_ran[key] = value
    
print ("Test  Train  Max_features")
for key in sorted(test_accuracy_ran, key=test_accuracy_ran.get, reverse=True):
    print ("{:.4f}, {:.4f}, {}"
          .format(test_accuracy_ran[key], training_accuracy_ran[key], key))

Test  Train  Max_features
0.0225, 0.9931, 15
0.0223, 0.9911, 14
0.0222, 0.9900, 13
0.0218, 0.9942, 16
0.0218, 0.9967, 20
0.0216, 0.9956, 18
0.0216, 0.9958, 19
0.0213, 0.9492, 7
0.0213, 0.9951, 17
0.0206, 0.9720, 9
0.0206, 0.9790, 10
0.0204, 0.9872, 12
0.0202, 0.9836, 11
0.0201, 0.9006, 5
0.0199, 0.9280, 6
0.0199, 0.9619, 8
0.0185, 0.8531, 4
0.0179, 0.6364, 2
0.0178, 0.7702, 3
0.0169, 0.6369, 1
